In [1]:
import duckdb
import pandas as pd

In [2]:
test_path = 'D:/Taggle/test.parquet'
train_path = 'D:/Taggle/train.parquet' #定义训练集和测试集的位置

con = duckdb.connect()  #连接到DuckDB

#从Parquet文件中读取数据
#f""".."""是python中格式化字符串（f-strings)的一种形式，允许在字符串中嵌入表达式，并在运行时求值和替换
#SELECT * 从数据源中选择所有列
#parquet_scan用于从指定路径的parquet文件中读取数据，{}为之前预定义的占位符，对应前面定义的训练集和测试集
#WHERE binds = 0 限制读取条件为binds列为0的所有行
#ORDER BY random() #随机排序这些行#
# LIMIT 30000) #限制返回的行数为30000行#
#  UNION ALL #将两部分结果合并在一起，保留所有重复的行#
#.df() #.df将query方法返回的结果通过df的方法转换为pandas dataframe便于进行后续的分析处理

df = con.query (f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0 
                        ORDER BY random() 
                        LIMIT 30000)
                        UNION ALL
                        (SELECT * 
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df() 

con.close() #关闭连接 

In [3]:
df.head() #查看数据框的前五行

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,56395586,Cc1ccc(C(CC(=O)O)NC(=O)OCC2c3ccccc3-c3ccccc32)cc1,NCc1ccnc(-n2cncn2)c1,Nc1ccc2nsnc2c1,Cc1ccc(C(CC(=O)N[Dy])Nc2nc(NCc3ccnc(-n4cncn4)c...,sEH,0
1,14496416,CC(C)(C)OC(=O)N1CCN(C(=O)OCC2c3ccccc3-c3ccccc3...,Cc1c(N)cccc1Br,Nc1cnc(Cl)cc1Cl,Cc1c(Br)cccc1Nc1nc(Nc2cnc(Cl)cc2Cl)nc(N2CCN(C(...,sEH,0
2,208317522,O=C(Nc1cccc(Br)c1C(=O)O)OCC1c2ccccc2-c2ccccc21,COCc1ccc(N)cc1,Cl.Cl.NCc1cccc(-n2ccnn2)c1,COCc1ccc(Nc2nc(NCc3cccc(-n4ccnn4)c3)nc(Nc3cccc...,BRD4,0
3,262189193,O=C(O)C[C@@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,N#Cc1ncc(N)cc1C(F)(F)F,Cl.NCCc1cc(Br)c(Br)s1,N#Cc1ncc(Nc2nc(NCCc3cc(Br)c(Br)s3)nc(N3CCC[C@H...,sEH,0
4,129787766,O=C(N[C@H]1CCC[C@@H]1C(=O)O)OCC1c2ccccc2-c2ccc...,CN1C(=O)Cc2cc(CN)ccc21.Cl,Cc1cc(Cl)nnc1N,Cc1cc(Cl)nnc1Nc1nc(NCc2ccc3c(c2)CC(=O)N3C)nc(N...,sEH,0


In [4]:
from rdkit import Chem  # RDKit的核心模块，用于分子对象的创建和基本操作
from rdkit.Chem import AllChem  # RDKit的拓展模块，提供高级化学功能
from sklearn.ensemble import RandomForestClassifier  # 随机森林分类器
from sklearn.model_selection import train_test_split  # 数据集划分
from sklearn.metrics import average_precision_score  # 评估指标
from sklearn.preprocessing import OneHotEncoder  # 一热编码


In [5]:
#将smile转为RDKit的分子
#df[]是pandas对数据框进行访问、修改的常见用法

df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles) #借助pandas中的apply方法，对数据框中molecule_smiles列调用Chem.MolFromSmiles，将结果存储在新的molecule列中

#定义了一个生成ECFP的函数“generate_ecfp",ECFP的半径为2（默认），生成的指纹位数为1024
#GetMorganFingerprintAsBitVect是RDKit提供的函数，用于生成分子的Morgan指纹（ECFP）
#这个函数式定义了一个ECFP生成函数，首先会读取molecule这一列，如果是空则返回none，否则就返回生成的指纹列表

def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits)) 
    
#根据generate_ecfp函数进行ECFPs的生成，并存储在列表的ecfp列中

df['ecfp'] = df['molecule'].apply(generate_ecfp)#得到ECFPs

In [6]:
df.head() #会发现多了两列，molecule和ecfp

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,molecule,ecfp
0,56395586,Cc1ccc(C(CC(=O)O)NC(=O)OCC2c3ccccc3-c3ccccc32)cc1,NCc1ccnc(-n2cncn2)c1,Nc1ccc2nsnc2c1,Cc1ccc(C(CC(=O)N[Dy])Nc2nc(NCc3ccnc(-n4cncn4)c...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x0000025005A...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,14496416,CC(C)(C)OC(=O)N1CCN(C(=O)OCC2c3ccccc3-c3ccccc3...,Cc1c(N)cccc1Br,Nc1cnc(Cl)cc1Cl,Cc1c(Br)cccc1Nc1nc(Nc2cnc(Cl)cc2Cl)nc(N2CCN(C(...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x0000025005A...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,208317522,O=C(Nc1cccc(Br)c1C(=O)O)OCC1c2ccccc2-c2ccccc21,COCc1ccc(N)cc1,Cl.Cl.NCc1cccc(-n2ccnn2)c1,COCc1ccc(Nc2nc(NCc3cccc(-n4ccnn4)c3)nc(Nc3cccc...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x0000025005A...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,262189193,O=C(O)C[C@@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,N#Cc1ncc(N)cc1C(F)(F)F,Cl.NCCc1cc(Br)c(Br)s1,N#Cc1ncc(Nc2nc(NCCc3cc(Br)c(Br)s3)nc(N3CCC[C@H...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x0000025005A...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,129787766,O=C(N[C@H]1CCC[C@@H]1C(=O)O)OCC1c2ccccc2-c2ccc...,CN1C(=O)Cc2cc(CN)ccc21.Cl,Cc1cc(Cl)nnc1N,Cc1cc(Cl)nnc1Nc1nc(NCc2ccc3c(c2)CC(=O)N3C)nc(N...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x0000025005A...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False) #将分类变量转化为一热编码，并指定输出为密集数组(非稀疏矩阵）
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1)) #将蛋白质名称转化为二维数组，从而符合OneHotEncoder的输入格式

# Combine ECFPs and one-hot encoded protein_name
#我们需要将所有的特征组合成单一的输入矩阵（或向量），以便将其传递给模型，在这里，我们有两组特征：ECFPS表示分子的化学结构特征、One-hot encoded protein_name表示蛋白质的信息特征
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
#利用train_test_split函数可以将数据集划分为训练集和测试集
#训练集和测试集的划定(test_size)需要根据数据集大小以及模型评估需求来确定，数据集小则需要考虑更大的训练集比例，一般为0.2
#划分数据集要设置随机种子以确保数据集划分的一致性和结果的可重复性，通常选择42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the random forest model
#n_estimators指定了随机森林中树的数量，此处包含100课决策树，更多的树意味着更稳定和更准确的模型，但是计算成本也会增加。
#使用'fit'的方法进行模型训练，让模型学会输入特征和目标标签之间的关系
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
#使用训练好的随机森林模型对测试数据集进行预测，并提取预测为正类（通常表示目标类别为1）的概率
#[:,1]是numpy中的切片操作，目的是提取每个样本属于类别1的概率
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision 计算并打印模型在测试集上的平均精度
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.96


In [8]:
#利用训练的模型进行预测
import os

# Process the test.parquet file chunk by chunk
test_file = 'D:/Taggle/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
#chunksize指定了分块读取，即每次读取100000行数据，对于大文件的读取，可以节省内存，对于每个分块进行下面的处理



#创建输出数据框并保存结果
#for循环使用pandas的read_csv函数结合chunksize参数，分块读取文件，生成器会逐块读取文件，直到迭代自动结束
for df_test in pd.read_csv(test_file, chunksize=100000):

    # Generate ECFPs for the molecule_smiles
    #生成分子指纹并且放置在新的一列数据中
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    #将蛋白进行一热编码
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    #将分子指纹与蛋白热编码进行组合，生成特征向量列表X_test
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    #使用随机森林模型对预测测试集进行预测，并计算测试集样本中属于类别1的概率
    probabilities = rf_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    #创建输出数据框
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    #将结果保存为csv文件
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
